In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 58625, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 58625 (delta 212), reused 291 (delta 159), pack-reused 58274
Receiving objects: 100% (58625/58625), 573.35 MiB | 36.10 MiB/s, done.
Resolving deltas: 100% (40690/40690), done.


In [3]:
%cd models/research/

/content/models/research


In [4]:
!git checkout 5f4d34fc

Checking out files: 100% (4045/4045), done.
Note: checking out '5f4d34fc'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 5f4d34fc Add kwargs to make the benchmark class constructor forward compatible. (#6246)


In [6]:
%cd slim

/content/models/research/slim


In [7]:
!pip install .

Processing /content/models/research/slim
  Created wheel for slim: filename=slim-0.1-cp37-none-any.whl size=205718 sha256=af2071aeb43c455f6c2cf0557146a2a4b57c67834ec9a05dd30a1d33990b0e84
  Stored in directory: /tmp/pip-ephem-wheel-cache-v_e0gyjp/wheels/c0/cb/18/e76d02033e40846187fac06f0f855c6434d52aadf580451f20
Successfully built slim


In [8]:
#edit some issue for python file in path datasets/imagenet.py 
%cd datasets/
!sed -i 's/filename, _ = urllib.request.urlretrieve(synset_url)/#filename, _ = urllib.request.urlretrieve(synset_url)/' imagenet.py
!sed -i 's/filename, _ = urllib.request.urlretrieve(synset_to_human_url)/#filename, _ = urllib.request.urlretrieve(synset_to_human_url)/' imagenet.py
!sed -i 's,https://raw.githubusercontent.com/tensorflow/models/master/research/inception/inception/data/,datasets,g' imagenet.py
!sed -i 's/s.strip() for s in open(filename).readlines()/s.strip() for s in open(synset_url).readlines()/' imagenet.py
!sed -i 's/synset_to_human_list = open(filename).readlines()/synset_to_human_list = open(synset_to_human_url).readlines()/' imagenet.py
%cd ..

/content/models/research/slim/datasets
/content/models/research/slim


In [12]:
#git clone tensorflow using python API tool convert to frozen_graph
!git clone -b r1.15 --single-branch https://github.com/tensorflow/tensorflow.git

Cloning into 'tensorflow'...
remote: Enumerating objects: 658383, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 658383 (delta 89), reused 89 (delta 87), pack-reused 658281
Receiving objects: 100% (658383/658383), 370.68 MiB | 28.87 MiB/s, done.
Resolving deltas: 100% (532760/532760), done.
Checking out files: 100% (20043/20043), done.


In [13]:
#vgg checkpoint form slim mdoel 
!wget http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz -q --show-progress --no-clobber
!wget http://download.tensorflow.org/models/vgg_19_2016_08_28.tar.gz -q --show-progress --no-clobber
!tar -xvf vgg_16_2016_08_28.tar.gz
!tar -xvf vgg_19_2016_08_28.tar.gz

vgg_16_2016_08_28.t 100%[===================>] 489.54M   126MB/s    in 4.1s    
vgg_19_2016_08_28.t 100%[===================>] 508.39M   138MB/s    in 3.7s    
vgg_16.ckpt
vgg_19.ckpt


In [10]:
#for vgg_16
!python export_inference_graph.py --alsologtostderr --model_name=vgg_16 --output_file=vgg16_inf.pb --labels_offset=1 #no Background set labels_offset=1
#for vgg_19
!python export_inference_graph.py --alsologtostderr --model_name=vgg_19 --output_file=vgg19_inf.pb --labels_offset=1 #no Background set labels_offset=1

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0710 08:47:10.977320 140650891732864 module_wrapper.py:139] From export_inference_graph.py:122: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0710 08:47:10.977511 140650891732864 module_wrapper.py:139] From export_inference_graph.py:122: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0710 08:47:10.981651 140650891732864 module_wrapper.py:139] From /content/models/research/slim/datasets/imagenet.py:151: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.


W0710 08:47:10.981877 1406508917328

In [14]:
!python tensorflow/tensorflow/python/tools/freeze_graph.py --input_graph vgg16_inf.pb --input_checkpoint vgg_16.ckpt --input_binary=true --output_graph=frozen_vgg16.pb --output_node_names vgg_16/fc8/squeezed
!python tensorflow/tensorflow/python/tools/freeze_graph.py --input_graph vgg19_inf.pb --input_checkpoint vgg_19.ckpt --input_binary=true --output_graph=frozen_vgg19.pb --output_node_names vgg_19/fc8/squeezed

Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0710 08:51:09.891719 140607987767168 deprecation.py:323] From tensorflow/tensorflow/python/tools/freeze_graph.py:127: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2021-07-10 08:51:09.924744: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-07-10 08:51:09.991635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-10 08:51:09.992241: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2021-07-10 08:51

In [15]:
#Download frozen graph vgg19 and vgg16
def download(path):
  from google.colab import files
  files.download(path)
download('frozen_vgg16.pb')
download('frozen_vgg19.pb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>